- create a dummy collection
- index a sample product using a logstash 

In [2]:
# %pip install rich -Uqq
# %pip install fastcore -Uqq
# %pip install nbdev -Uqq
# %pip install opensearch-py -Uqq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from rich import inspect as rinspect
from rich import print as rprint
from fastcore.test import *
from fastcore.net import *
from fastcore.basics import *

In [2]:
from opensearchpy import OpenSearch
import json
host, port = 'localhost', 9200
auth=('admin', 'admin')

In [3]:
cli=OpenSearch(hosts=[{'host': host, 'port': port}], http_compress=True, http_auth=auth, use_ssl=True, verify_certs=False, ssl_assert_hostname=False, ssl_show_warn=False)

In [4]:
cli.cat.health(v=True, h=['status'])
cli.cat.indices(v=True, index='searchml_test')

'health status index         uuid                   pri rep docs.count docs.deleted store.size pri.store.size\nyellow open   searchml_test qaLnnjpdSMqrLN628eCASA   1   1          4            0     21.4kb         21.4kb\n'

```
   <productId>1051806942127</productId>
    <name>Fuji - 110 Color Print Disc Film (24 exposures)</name>
    <source>bestbuy</source>
    <type>HardGood</type>
    <startDate>2000-03-01</startDate>
    <new>false</new>
    <active>false</active>
    <activeUpdateDate>2012-07-02T22:28:55</activeUpdateDate>
    <regularPrice>1.89</regularPrice>
    <salePrice>1.89</salePrice>
    <onSale>false</onSale>
    
```

In [5]:
# You should have one field mapping entry for every field specified in the index-bbuy.logstash XPath filter. 
# All fields that are type “text” should also have a “keyword” multi-field.
# References: https://www.elastic.co/guide/en/elasticsearch/reference/8.0/mixing-exact-search-with-stemming.html

index_name='bbuy_products'
default_index_body = {
  "settings": {
    "index.refresh_interval": "5s"
  }
}

index_body={
  "settings": {
    "index.refresh_interval": "5s"
  },
  "analysis": {
      "analyzer": {
        "english_exact": {
          "tokenizer": "standard",
          "filter": [
            "lowercase"
          ]
        }
      }
    },
  "mappings": {
      "properties": {
          "productId": {"type": "keyword"}, # structured content such as IDs
          "sku": {"type": "keyword"}, # we may want to change to numeric if sku needs to be used in range queries
          "name": {'type': 'text', 'analyzer': 'english', # TODO stemming
            'fields': {
              'keyword': {'type': 'keyword', 'ignore_above': "256"}
            }
          },
          "type": {"type": "keyword", 'ignore_above': "256"}, # better for term queries
          "startDate": {'type': 'date'},
          'active': {'type': 'boolean'},
          'new': {'type': 'boolean'},          
          'regularPrice': {
            'type': 'float',
            'fields': {
              "range": {"type": "float_range"}
            }
          },
          'salePrice': {'type': 'float'},
          'onSale': {'type': 'boolean'},
          'digital': {'type': 'boolean'},
          #"frequentlyPurchasedWith": {"type": "keyword", 'ignore_above': "256"}, # TODO this can contain subfields, dynamicTemplates may be??? 
          # "accessories": {"type": "keyword", 'ignore_above': "256"}, # TODO this can contain subfields
          # "relatedProducts": {"type": "keyword", 'ignore_above': "256"}, # TODO this can contain subfields
          'crossSell': {'type': 'keyword'},
          'freeShipping': {'type': 'boolean'},
          'salesRankShortTerm': {'type': 'integer'},
          'salesRankMediumTerm': {'type': 'integer'},
          'salesRankLongTerm': {'type': 'integer'},
          'bestSellingRank': {'type': 'integer'},
          'url': {'type': 'keyword'},

          'inStoreAvailability': {'type': 'boolean'},   
          'shortDescription': {'type': 'text', 'analyzer': 'english'}, # use multi-fields 
          'longDescription': {'type': 'text', 'analyzer': 'english'},
          'department': {"type": "keyword", 'ignore_above': "256"},
          "class": {"type": "keyword"},
          "classId": {"type": "long"},
          "subclass": {"type": "keyword"},
          "department": {"type": "keyword"},
          "departmentId": {"type": "long"},
          "bestBuyItemId": {"type": "long"},

        }
    }
}
cli.indices.create(index=index_name, body=default_index_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bbuy_products'}

In [34]:
cli.cat.indices(v=True, index='bbuy_products', h=['docs.count'])

'docs.count\n         0\n'

In [28]:
# cli.indices.delete(index='bbuy_products')

{'acknowledged': True}

In [31]:
# index sample data using logstash
# Make sure you change index-bbuy.logstash input path setting from sample_products to products
!../../logstash/logstash-7.13.2/bin/logstash -f ../logstash/index-bbuy.logstash

Using JAVA_HOME defined java: /home/gitpod/.sdkman/candidates/java/current
WARNING, using JAVA_HOME while Logstash distribution comes with a bundled JDK
OpenJDK 64-Bit Server VM warning: Option UseConcMarkSweepGC was deprecated in version 9.0 and will likely be removed in a future release.
Sending Logstash logs to /workspace/logstash/logstash-7.13.2/logs which is now configured via log4j2.properties
[2022-02-16T11:51:25,609][INFO ][logstash.runner          ] Log4j configuration path used is: /workspace/logstash/logstash-7.13.2/config/log4j2.properties
[2022-02-16T11:51:25,618][INFO ][logstash.runner          ] Starting Logstash {"logstash.version"=>"7.13.2", "jruby.version"=>"jruby 9.2.16.0 (2.5.7) 2021-03-03 f82228dc32 OpenJDK 64-Bit Server VM 11.0.13+8-LTS on 11.0.13+8-LTS +indy +jit [linux-x86_64]"}
[2022-02-16T11:51:25,987][WARN ][logstash.config.source.multilocal] Ignoring the 'pipelines.yml' file because modules or command line options are specified
[2022-02-16T11:51:27,333][INFO

In [ ]:
resp=cli.search(index='bbuy_products', params={'q': '*'})

**Query String** Returns documents based on a provided query string, using a parser with a strict syntax. Recommendation: If you don’t need to support a query syntax, consider using the match query. If you need the features of a query syntax, use the simple_query_string query, which is less strict.

```
GET /_search
{
  "query": {
    "query_string": {
      "query": "(new york city) OR (big apple)",
      "default_field": "content"
    }
  }
}
```

**Simple query string**
Returns documents based on a provided query string, using a parser with a limited but fault-tolerant syntax.

This query uses a simple syntax to parse and split the provided query string into terms based on special operators. The query then analyzes each term independently before returning matching documents.

While its syntax is more limited than the query_string query, the simple_query_string query does not return errors for invalid syntax. Instead, it ignores any invalid parts of the query string.

```
GET /_search
{
  "query": {
    "simple_query_string" : {
        "query": "\"fried eggs\" +(eggplant | potato) -frittata",
        "fields": ["title^5", "body"],
        "default_operator": "and"
    }
  }
}
```

The simple_query_string query supports the following operators:

+ signifies AND operation
| signifies OR operation
- negates a single token
" wraps a number of tokens to signify a phrase for searching
* at the end of a term signifies a prefix query
( and ) signify precedence
~N after a word signifies edit distance (fuzziness)
~N after a phrase signifies slop amount

In [3]:
#%pip install pandas -Uqq

In [4]:
import pandas as pd

In [13]:
df = pd.read_csv('../../datasets/train.csv', parse_dates=['query_time', 'click_time'], dtype={'query': str, 'category': str, 'sku': str})

In [14]:
df.columns, df.dtypes

(Index(['user', 'sku', 'category', 'query', 'click_time', 'query_time'], dtype='object'),
 user                  object
 sku                   object
 category              object
 query                 object
 click_time    datetime64[ns]
 query_time    datetime64[ns]
 dtype: object)

In [15]:
df.head()

,user,sku,category,query,click_time,query_time
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233,abcat0101001,Televisiones Panasonic 50 pulgadas,2011-09-01 23:44:52.533,2011-09-01 23:43:59.752
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324,abcat0101001,Sharp,2011-09-05 12:25:37.420,2011-09-05 12:25:01.187
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,1517163,pcmcat193100050014,nook,2011-08-24 12:56:58.910,2011-08-24 12:55:13.012
3,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125,abcat0101001,rca,2011-10-25 07:18:14.722,2011-10-25 07:16:51.759
4,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877134,abcat0101005,rca,2011-10-25 07:19:51.697,2011-10-25 07:16:51.759


In [ ]:
df.to_json('../../datasets/train.json', orient='records')